<a href="https://colab.research.google.com/github/hamdiguedhami/hamdiguedhami/blob/main/FULL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, LSTM, Dropout, Dense, Flatten
from tensorflow.keras.utils import to_categorical
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # Ajout de tqdm pour la barre de progression

# Charger les données
data = pd.read_csv("/content/WSN-DS.csv")

# Nettoyer les noms de colonnes en supprimant les espaces
data.columns = data.columns.str.strip()

# Afficher les noms des colonnes
print("Noms des colonnes:", data.columns)

# Suppression des valeurs nulles et des doublons
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

# Encodage des étiquettes
encoder = OneHotEncoder()
labels = encoder.fit_transform(data[['Attack type']]).toarray()

# Normalisation des données avec MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.drop(columns=['id', 'Attack type'], errors='ignore'))

# Sélection des caractéristiques
selector = SelectKBest(score_func=chi2, k=10)
selected_features = selector.fit_transform(scaled_data, labels.argmax(axis=1))

# Ajouter une dimension supplémentaire pour Conv1D
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(selected_features, labels, test_size=0.2, random_state=42)

# Fonction pour créer le modèle
def create_model(neurons_1, neurons_2, neurons_3, dropout_rate):
    model = Sequential()
    model.add(Conv1D(filters=neurons_1, kernel_size=1, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(LSTM(units=neurons_1, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(filters=neurons_2, kernel_size=1, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(LSTM(units=neurons_2, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(filters=neurons_3, kernel_size=1, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(LSTM(units=neurons_3, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(units=labels.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Paramètres spécifiques pour cette partie de l'entraînement
neurons_1 = 16
neurons_2 = 32
neurons_3 = 64
dropout_options = [0.2, 0.5]
k_values = [2, 3, 4, 5, 6, 7, 8, 9, 10]

# Stockage des résultats
results = []

# Entraînement et évaluation du modèle
for dropout_rate, k in tqdm(itertools.product(dropout_options, k_values), desc="Total Progress", total=len(dropout_options)*len(k_values)):
    skf = StratifiedKFold(n_splits=k)
    for train_index, val_index in tqdm(skf.split(X_train, y_train.argmax(axis=1)), desc=f"Dropout: {dropout_rate}, K: {k}", leave=False):
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, y_val = y_train[train_index], y_train[val_index]
        model = create_model(neurons_1, neurons_2, neurons_3, dropout_rate)
        model.fit(X_tr, y_tr, epochs=5, batch_size=32, verbose=0)
        y_pred = model.predict(X_val)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true_classes = np.argmax(y_val, axis=1)
        acc = accuracy_score(y_true_classes, y_pred_classes)
        precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
        recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
        f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
        conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
        fp = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
        fn = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
        tp = np.diag(conf_matrix)
        tn = conf_matrix.sum() - (fp + fn + tp)
        fp = fp.astype(float)
        fn = fn.astype(float)
        tp = tp.astype(float)
        tn = tn.astype(float)
        far = fp / (fp + tn)
        dr = tp / (tp + fn)
        results.append((3, [16, 32, 64], dropout_rate, k, acc, precision, recall, f1, far.mean(), dr.mean()))

# Afficher les résultats sous forme de tableau
results_df = pd.DataFrame(results, columns=['Layers', 'Neurons', 'Dropout Rate', 'K', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'FAR', 'DR'])
display(results_df)

# Générer des plots
fig, axs = plt.subplots(2, 3, figsize=(18, 12))

sns.boxplot(x='K', y='Accuracy', data=results_df, ax=axs[0, 0])
axs[0, 0].set_title('Accuracy by K')

sns.boxplot(x='K', y='Precision', data=results_df, ax=axs[0, 1])
axs[0, 1].set_title('Precision by K')

sns.boxplot(x='K', y='Recall', data=results_df, ax=axs[0, 2])
axs[0, 2].set_title('Recall by K')

sns.boxplot(x='K', y='F1 Score', data=results_df, ax=axs[1, 0])
axs[1, 0].set_title('F1 Score by K')

sns.boxplot(x='K', y='FAR', data=results_df, ax=axs[1, 1])
axs[1, 1].set_title('False Alarm Rate (FAR) by K')

sns.boxplot(x='K', y='DR', data=results_df, ax=axs[1, 2])
axs[1, 2].set_title('Detection Rate (DR) by K')

plt.tight_layout()
plt.show()


Noms des colonnes: Index(['id', 'Time', 'Is_CH', 'who CH', 'Dist_To_CH', 'ADV_S', 'ADV_R',
       'JOIN_S', 'JOIN_R', 'SCH_S', 'SCH_R', 'Rank', 'DATA_S', 'DATA_R',
       'Data_Sent_To_BS', 'dist_CH_To_BS', 'send_code', 'Expaned Energy',
       'Attack type'],
      dtype='object')


Total Progress:   0%|          | 0/18 [00:00<?, ?it/s]
Dropout: 0.2, K: 2: 0it [00:00, ?it/s]

4573/4573 [==============================] - 17s 4ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

Dropout: 0.2, K: 2: 1it [03:59, 239.19s/it]

4573/4573 [==============================] - 17s 3ms/step


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

Dropout: 0.5, K: 8: 3it [19:50, 396.80s/it]

1144/1144 [==============================] - 6s 4ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

Dropout: 0.5, K: 8: 4it [28:26, 444.05s/it]

1144/1144 [==============================] - 7s 5ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

Dropout: 0.5, K: 8: 5it [37:08, 471.98s/it]